# Autonomous driving with voice command  


## Load the trained model

Upload the model into this notebook's directory by using the Jupyter Lab upload tool. Once that's finished there should be a file named best_model.pth in this notebook's directory 


Execute the code below to initialize the PyTorch model.

In [0]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 4)

# Alexnet was introduced in the paper ImageNet Classification with Deep Convolutional Neural Networks 
# and was the first very successful CNN on the ImageNet dataset. 
# When we print the model architecture, we see the model output comes from the 6th layer of the classifier
# For more information, please see https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html 

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [0]:
model.load_state_dict(torch.load('best_model.pth'))

Currently, the model weights are located on the CPU memory, so you should execute the code below to transfer the model to the GPU device.

In [0]:
device = torch.device('cuda')
model = model.to(device)

## Create the preprocessing function

You have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
You need to do some *preprocessing*.  This involves the following steps: 

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [0]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [0]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image]))

You'll also create your robot instance which you'll need to drive the motors.

In [0]:
from jetbot import Robot

robot = Robot()

## Autonomous driving logic

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

The following chart shows the logic of robot movement given the probabilities predicted by the models. and This logic refers to the example provided by [Dmitri Villevald](https://www.hackster.io/dvillevald/transfer-learning-with-nvidia-jetbot-fun-with-cones-adf531)



<img src='https://ifh.cc/g/onOTpZ.png' width='800'>

In [0]:
import torch.nn.functional as F
import time

# Simple PD controller (Kp - proportional term, Kd - derivative term)
Kp = 0.18
Kd = 0.05

frwd_value = 0.3                      # Default value to drive forward (0 = no action, 1 = full motor capacity)
rot_value_when_exploring = 0.3        # Default value to spin to the right when robot is in exploration mode (0 = no action, 1 = full motor capacity)
min_prob_free_to_drive_frwd = 0.25    # Min probability prob_free for robot to drive forward 
max_n_frames_stuck = 20               # Limit on the number of frames the robot is stuck for. Once this limit is reached, robot goes into exploration mode (makes large right turn)
frame_counter = 0                     # Frame counter 
n_frames_stuck = 0                    # Initialize counter of the number of successive frames the robot is stuck for
exploration = False                   # Initialize binary variable which determines if robot is in exploration mode (when True.) Used to mark the related frames with red background  
recent_detections = []                # Initialize the array to store the last frame data

def update(change):
    global robot, frame_counter, n_frames_stuck, exploration
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    y = y.flatten()
   
    # extract probabilities of blocked, free, left and right
    prob_blocked = float(y[0])
    prob_free = float(y[1])
    prob_left = float(y[2])
    prob_right = float(y[3])
 
    # update list of recent detections
    while (len(recent_detections) >= 2):
        recent_detections.pop(0)
    recent_detections.append([prob_free,prob_left,prob_right,prob_blocked])
    
    # check if robot got stuck and update n_frames_stuck counter
    if prob_free < min_prob_free_to_drive_frwd:  # min_prob_free_to_drive_frwd = 0.25
        n_frames_stuck = n_frames_stuck + 1 
    else:
        n_frames_stuck = 0
        
    # calculate errors at times t (current) and t-1 (prev)    
    # error(t) and error(t-1): prob_left-prob_right   
    if len(recent_detections) == 2:
        current_probs = recent_detections[1]
        prev_probs = recent_detections[0]
    else:
        current_probs = [prob_free,prob_left,prob_right,prob_blocked]
        prev_probs = current_probs
                
    # error = prob_left-prob_right        
    current_error = current_probs[1] - current_probs[2]
    prev_error    = prev_probs[1] - prev_probs[2]

    # increment frame counter 
    frame_counter = frame_counter + 1
    
    # define functions which deterine (and return) robot actions
    def forward(value):
        robot.forward(value)
        return ("FWRD",round(value,2))

    def left(value):
        robot.left(value)
        return ("LEFT",round(value,2))

    def right(value):
        robot.right(value)
        return ("RGHT",round(value,2))
    
    action = ""
  
    # estimate rotational value to turn left (if negative) or right (if positive)
    # 0 = no action, 1 = full motor capacity)
    rot_value = - Kp * current_error - Kd * (current_error - prev_error)
    
    # store propotional and differential controller components for frame-by-frame analysis
    p_component = - Kp * current_error
    d_component = - Kd * (current_error - prev_error)
    
    # initalize binary flag showinf if robot rotates 
    robot_rotates = False
    
    # action logic
    # moving forward if there is no obstacles
    if prob_free > min_prob_free_to_drive_frwd:
        action = forward(frwd_value)

    # turn left or right if robot is not blocked for a long time
    elif n_frames_stuck < max_n_frames_stuck:
        robot_rotates = True
        if rot_value < 0.0:
            action = left(-rot_value)
        else:
            action = right(rot_value)

    # activate exploration mode - robot turns right by a large (45-90 degree) angle if it failed to move forward for [max_n_frames_stuck] recent frames
    else:
        exploration = True
        robot_rotates = True
        action = right(rot_value_when_exploring)
        time.sleep(0.5)
        n_frames_stuck = 0
    
    time.sleep(0.001)


    # append frame's telemetry and robot action to the stored data 
    if not robot_rotates:
        rot_value = 0.
        p_component = 0.
        d_component = 0.
    if robot_rotates and exploration:
        rot_value = rot_value_when_exploring
        p_component = 0.
        d_component = 0.
    
    # reset variables
    exploration = False
    robot_rotates = False

In [0]:
update({'new': camera.value})  
robot.stop()

You've created our neural network execution function, but now you need to attach it to the camera for processing. 

You accomplish that with the observe function.


In [0]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Great! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches the cones.

To stop this behavior, unattach this callback by executing the code below.

In [0]:
camera.unobserve(update, names='value')
update({'new': camera.value}
robot.stop()

(Optional) Perhaps you want the robot to run without streaming video to the browser.  You can unlink the camera as below.

In [0]:
camera_link.unlink()  # don't stream to browser (will still run camera)

To continue streaming, call the following.

In [0]:
camera_link.link()  # stream to browser (wont run camera)

## Autonomous driving with human intervention
We not only let JetBot drive automatically through image processing, but also allow us to control it during autonomous driving through voice commands. 

If your robot is well trained, it will be easy for your robot to find way. However, your robot may encounter an unlearned situation while finding its way. For example, your robot may be facing a traffic light, at risk of bumping into another robot, or at a crossroads. In that case, you have to send the command to the robot to solve the situation.


Therefore, we presented the following simple scenario.

<img src='https://ifh.cc/g/vCeRTQ.png' width='500'>


>Example

><img src='https://ifh.cc/g/fmN9ru.gif' width='800'>

To achieve these, we made the same logic as follows:

<img src='https://ifh.cc/g/WkyNF3.png' width='750'>

In [0]:
import robotdecoder as snowboydecoder
import sys
import signal
from jetbot import Robot
import time

# Demo code for listening to four hotwords at the same time

robot = Robot()
interrupted = False

def signal_handler(signal, frame):
    global interrupted
    interrupted = True


def interrupt_callback():
    global interrupted
    return interrupted

# To run the model in terminal, you need to enter the following command
# python3 robotmove.py forward.pmdl stop.pmdl right.pmdl left.pmdl
# The sys.argv stores: [robotmove.py, forward.pmdl, stop.pmdl, right.pmdl, left.pmdl]

if len(sys.argv) != 5:
    print("Error: need to specify 4 model names")
    sys.exit(-1)


models = sys.argv[1:]
# models = [forward.pmdl, stop.pmdl, right.pmdl, left.pmdl]

# initialize
update({'new': camera.value})  
robot.stop()
# You need to set the logic for your command
def wakeup():
    camera.observe(update, names='value')

def stop():
    camera.unobserve(update, names='value')
    update({'new': camera.value}
    robot.stop()

def left():
    # Turn left for 1 second
    robot.left(0.3)
    time.sleep(1)
    robot.stop()

    # Start autonomous driving
    camera.observe(update, names='value')

def right():
    # Turn right for 1 second
    robot.right(0.3)
    time.sleep(1)
    robot.stop()

   # Start autonomous driving
    camera.observe(update, names='value')
# capture SIGINT signal, e.g., Ctrl+

signal.signal(signal.SIGINT, signal_handler)
sensitivity = [0.5]*len(models)
detector = snowboydecoder.HotwordDetector(models, sensitivity=sensitivity)
# In order to work properly, you must match the input and your logic sequence in the command.
callbacks = [lambda: wakeup(),
             lambda: stop(),
             lambda: left(),
             lambda: right()]

print('Listening... Press Ctrl+C to exit')

# main loop
# make sure you have the same numbers of callbacks and models
detector.start(detected_callback=callbacks,
               interrupt_check=interrupt_callback,
               sleep_time=0.03)

detector.terminate()

You can run it from the terminal using the following command. (You should save the notebook file as ```robotmove.py```)

In [0]:
 $ python3 robotmove.py wakeup.pmdl stop.pmdl right.pmdl left.pmdl

You can also run it on Jupyter notebooks. To do this, you need to:

* Move pmdl files in the same directory as the running Jupyter file

In [0]:
import robotdecoder as snowboydecoder
import sys
import signal
from jetbot import Robot
import time

# Demo code for listening to four hotwords at the same time

robot = Robot()
interrupted = False

def signal_handler(signal, frame):
    global interrupted
    interrupted = True


def interrupt_callback():
    global interrupted
    return interrupted


models = ['wakeup.pmdl', 'stop.pmdl', 'right.pmdl', 'left.pmdl']

# initialize
update({'new': camera.value})  
robot.stop()
# You need to set the logic for your command
def wakeup():
    camera.observe(update, names='value')

def stop():
    camera.unobserve(update, names='value')
    update({'new': camera.value}
    robot.stop()

def left():
    # Turn left for 1 second
    robot.left(0.3)
    time.sleep(1)
    robot.stop()

    # Start autonomous driving
    camera.observe(update, names='value')

def right():
    # Turn right for 1 second
    robot.right(0.3)
    time.sleep(1)
    robot.stop()

   # Start autonomous driving
    camera.observe(update, names='value')
# capture SIGINT signal, e.g., Ctrl+

signal.signal(signal.SIGINT, signal_handler)
sensitivity = [0.5]*len(models)
detector = snowboydecoder.HotwordDetector(models, sensitivity=sensitivity)
# In order to work properly, you must match the input and your logic sequence in the command.
callbacks = [lambda: wakeup(),
             lambda: stop(),
             lambda: left(),
             lambda: right()]

print('Listening... Press Ctrl+C to exit')

# main loop
# make sure you have the same numbers of callbacks and models
detector.start(detected_callback=callbacks,
               interrupt_check=interrupt_callback,
               sleep_time=0.03)

detector.terminate()